# Machine Learning: CNN Architectures

## CNNs for Denoising

1. Unet with residual skip connections
2. Writing our own loss function

### MNIST Dataset

Imports

In [ ]:
import os
import datetime
import tensorflow as tf

Downloading the dataset (already splitted into train and test)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()


In [ ]:
print(train_images.shape, train_labels.shape)

In [ ]:
# image preprocessing
NB_IMAGES_TO_USE = 1000

train_images = train_images[:NB_IMAGES_TO_USE] / 255.0
test_images = test_images[:NB_IMAGES_TO_USE]  / 255.0

In [ ]:
print(train_images.shape, test_images.shape)

### Plot images with random noise

In [ ]:
import pylab as pl
import numpy as np

In [ ]:
pl.subplot(121)
pl.imshow(train_images[0], cmap=pl.cm.gray) 
pl.xticks(())
pl.yticks(())
pl.title("Original")
pl.subplot(122)
pl.imshow(train_images[0] + np.random.normal(0., 0.1, (28, 28)), cmap=pl.cm.gray)
pl.xticks(())
pl.yticks(())
pl.title("Noised (Input to the network)")
#pl.show()

## Image Generators

In [ ]:
def generator(images):
    while True:
        for im in images:
            noised = im + np.random.normal(0., 0.1, im.shape)
            noised = noised[np.newaxis]
            yield noised, im[np.newaxis]

In [ ]:
train_gen = generator(train_images)

In [ ]:
a, b = next(train_gen)
a, b = next(train_gen)
print("Input shape", a.shape)
print("Output shape", b.shape)
pl.imshow(np.squeeze(b))

## Residual Networks

### The Residual Unit and Example

<img src="images/residulal_unit_and_exple.png" width="70%" height="30%"> 

### Our Previous UNet with Residual Skip Connections

<img src="images/unet_residual_skips.png" width="70%" height="30%"> 

### Coding: UNet with Residual Skip Connections

In [ ]:
# Encoder part

inputs = tf.keras.layers.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(8, (3,3), activation="relu", padding="same")(inputs)
l1 = tf.keras.layers.Conv2D(8, (3,3), activation="relu", padding="same")(x)
x = tf.keras.layers.Conv2D(16, (3,3), activation="relu", padding="same", strides=2)(l1) 
l2 = tf.keras.layers.Conv2D(16, (3,3), activation="relu", padding="same")(x)
x = tf.keras.layers.Conv2D(32, (3,3), activation="relu", padding="same", strides=2)(l2) 
x = tf.keras.layers.Conv2D(32, (3,3), activation="relu", padding="same")(x)

# Decoder part
x = tf.keras.layers.Conv2DTranspose(16, (3,3), activation="relu", padding="same", strides=2)(x)

x = tf.keras.layers.Add()([x, l2])

x = tf.keras.layers.Conv2D(16, (3,3), activation="relu", padding="same")(x)
x = tf.keras.layers.Conv2DTranspose(8, (3,3), activation="relu", padding="same", strides=2)(x)

x = tf.keras.layers.Add()([x, l1])

x = tf.keras.layers.Conv2D(8, (3,3), activation="relu", padding="same")(x)

# Output Layer
x = tf.keras.layers.Conv2D(1, (3,3), activation="sigmoid", padding="same")(x)

### Custom loss

In [ ]:
def bce(gt, pred):
    return -  gt * tf.math.log(pred) - (tf.ones_like(gt) - gt)*  tf.math.log(tf.ones_like(pred) - pred)

### Create the model and compile with the custom loss

In [ ]:
model = tf.keras.Model(inputs, x) 
model.compile(loss=bce, optimizer="adam")

In [ ]:
model.summary()

### Training

In [ ]:
# define a folder to store the training data for monitoring
logdir = os.path.join("resnet_logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# give the previous folder to Tensorboard 
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

train_gen = generator(train_images[:,:,:,np.newaxis])
model.fit(train_gen, epochs=10, steps_per_epoch=20, callbacks=[tensorboard_callback])

### Testing

In [ ]:
test_im = test_images[5][np.newaxis, :,:, np.newaxis] + np.random.normal(0., 0.1, (1, 28, 28, 1))

In [ ]:
outputs = model.predict(test_im, verbose=1)

In [ ]:
len(model(test_im))

In [ ]:
test_im.shape

#### Compute the loss on the test image

In [ ]:
# the ground-truth is the actual image
gt  = test_images[5][np.newaxis, :,:, np.newaxis].astype(np.float32) 
# computing the custom binary cross-entropy loss
l = tf.reduce_mean(bce(gt, outputs))
l.numpy() #unet - 0.037616905

In [ ]:
pl.subplot(121)
pl.imshow(np.squeeze(test_im))
pl.subplot(122)
pl.imshow(np.squeeze(outputs))

### Going further
1. Study the robustness by tweaking the noise
2. Add more layers 
 - Increase the layers on each levels
 - **Add residual connections between layers of the same levels**
 - Increase the levels 3-4-5
3. Try on other datasets